# Creation of our own model

In [1]:
import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.nn.functional as F
import matplotlib.pylab as plt
import numpy as np

import cv2
torch.manual_seed(2)

# https://www.kaggle.com/datasets/sshikamaru/car-object-detection/data

In [2]:
# Create the model class using sigmoid as the activation function

class CarRecognitionNet(nn.Module):
    def __init__(self, in_size):
        super(CarRecognitionNet, self).__init__()
        # self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        # self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(in_size, 128)  # Modifiez selon la taille de votre image
        self.fc2 = nn.Linear(128, 4)  # 4 pour les coordonnées (x_min, y_min, x_max, y_max)

    def forward(self, x):
        #x = torch.relu(self.conv1(x))
        #x = torch.max_pool2d(x, kernel_size=2)
        #x = torch.relu(self.conv2(x))
        #x = torch.max_pool2d(x, kernel_size=2)
        x = x.view(x.size(0), -1)  # Aplatir
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)  # Prédictions des coordonnées
        return x

In [3]:
# Train the model

def train(model, criterion, train_loader, test_loader, optimizer, epochs=10):
    i = 0
    output = {'training_loss': [], 'test_accuracy': []}  
    
    for epoch in range(epochs):
        for i, (x, y) in enumerate(train_loader):
            optimizer.zero_grad()
            print(x.shape)
            z = model(x.view(-1, 128 * 128 * 3))
            loss = criterion(z, y)
            loss.backward()
            optimizer.step()
            output['training_loss'].append(loss.data.item())
        
        correct = 0
        for x, y in test_loader:
            z = model(x.view(-1, 28 * 28))
            _, label = torch.max(z, 1)
            correct += (label == y).sum().item()
    
        accuracy = 100 * (correct / len(test_loader))
        output['test_accuracy'].append(accuracy)
    
    return output

In [4]:
import os
import json
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import torchvision.transforms as transforms
from skimage import io
import pandas as pd


class CarRecognitionDataset(Dataset):
    def __init__(self, images_dir, csv_annotations, transform=None):
        self.images_dir = images_dir
        self.annotations = pd.read_csv(csv_annotations)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.images_dir, self.annotations.iloc[index, 0])
        image = cv2.imread(img_path)
        image = image[:,:,:3] # removing alpha cannal if it exists
        image_resized = cv2.resize(image, (128, 128))
        image_tensor = torch.from_numpy(image_resized).permute(2, 0, 1).float() # readapte les canaux par permute
        
        y = torch.tensor(self.annotations.iloc[index, 1])

        if self.transform:
            image_trasnformed = self.transform(image_resized)
            image_trasnformed = torch.from_numpy(image_resized).permute(2, 0, 1).float()
            return image_trasnformed, y

        return image_tensor, y

        
        



In [5]:
dataset = CarRecognitionDataset(csv_annotations='data/train_boxes.csv', images_dir='data/train/',
                                 transform = transforms.ToTensor())

In [6]:
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [7]:
# for a,b in train_loader:
#     print(a)
#     print(b)

In [8]:
import torch.optim as optim

# images INITIAL shape : 676 x 380

learning_rate = 0.001
epochs = 10
model = CarRecognitionNet(128*128*3) # image size et 32 canaux dans le derneir filtre

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

output=train(model, criterion, train_loader, val_loader, optimizer, epochs)




torch.Size([32, 3, 128, 128])


c:\Users\Paul\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (4) must match the size of tensor b (32) at non-singleton dimension 1